<a href="https://colab.research.google.com/github/ZzangJay/team_project/blob/jangjaewon/20220917_dataprep_inclusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 환경설정

In [2]:
import pandas as pd
import numpy as np
#import tensorflow as tf 
import os
import sys
import sqlite3

In [3]:
from google.colab import drive  # for colab
drive.mount('/content/gdrive/')  # for colab

Mounted at /content/gdrive/


In [4]:
cloud_directory = '/content/gdrive/MyDrive/mimiciv_unzip/'
os.listdir(cloud_directory) # for colab, check cloud directory mount

['CHANGELOG.txt', 'LICENSE.txt', 'SHA256SUMS.txt', 'index.html', 'icu', 'hosp']

In [16]:
# labevents = pd.read_csv('/content/gdrive/MyDrive/mimiciv_unzip/hosp/labevents.csv')
# patients = pd.read_csv('/content/gdrive/MyDrive/mimiciv_unzip/hosp/patients.csv')
# d_labitems = pd.read_csv('/content/gdrive/MyDrive/mimiciv_unzip/hosp/d_labitems.csv')
# diagnosis = pd.read_csv('/content/gdrive/MyDrive/mimiciv_unzip/hosp/diagnosis.csv')

## 데이터 업로드(SQL)

### sql 함수 

In [18]:
def pd_to_sqlDB(input_df: pd.DataFrame,
                table_name: str,
                db_name: str = 'default.db') -> None:

    '''Take a Pandas dataframe `input_df` and upload it to `table_name` SQLITE table
    Args:
        input_df (pd.DataFrame): Dataframe containing data to upload to SQLITE
        table_name (str): Name of the SQLITE table to upload to
        db_name (str, optional): Name of the SQLITE Database in which the table is created. 
                                 Defaults to 'default.db'.
    '''

    # Step 1: Setup local logging
    import logging
    logging.basicConfig(level=logging.INFO,
                        format='%(asctime)s %(levelname)s: %(message)s',
                        datefmt='%Y-%m-%d %H:%M:%S')

    # Step 2: Find columns in the dataframe
    cols = input_df.columns
    cols_string = ','.join(cols)
    val_wildcard_string = ','.join(['?'] * len(cols))

    # Step 3: Connect to a DB file if it exists, else crete a new file
    con = sqlite3.connect(db_name)
    cur = con.cursor()
    logging.info(f'SQL DB {db_name} created')

    # Step 4: Create Table
    sql_string = f"""CREATE TABLE {table_name} ({cols_string});"""
    cur.execute(sql_string)
    logging.info(f'SQL Table {table_name} created with {len(cols)} columns')

    # Step 5: Upload the dataframe
    rows_to_upload = input_df.to_dict(orient='split')['data']
    sql_string = f"""INSERT INTO {table_name} ({cols_string}) VALUES ({val_wildcard_string});"""    
    cur.executemany(sql_string, rows_to_upload)
    logging.info(f'{len(rows_to_upload)} rows uploaded to {table_name}')
  
    # Step 6: Commit the changes and close the connection
    con.commit()
    con.close()

In [19]:
def sql_query_to_pd(sql_query_string: str, db_name: str ='default.db') -> pd.DataFrame:
    '''Execute an SQL query and return the results as a pandas dataframe
    Args:
        sql_query_string (str): SQL query string to execute
        db_name (str, optional): Name of the SQLITE Database to execute the query in.
                                 Defaults to 'default.db'.
    Returns:
        pd.DataFrame: Results of the SQL query in a pandas dataframe
    '''    
    # Step 1: Connect to the SQL DB
    con = sqlite3.connect(db_name)

    # Step 2: Execute the SQL query
    cursor = con.execute(sql_query_string)

    # Step 3: Fetch the data and column names
    result_data = cursor.fetchall()
    cols = [description[0] for description in cursor.description]

    # Step 4: Close the connection
    con.close()

    # Step 5: Return as a dataframe
    return pd.DataFrame(result_data, columns=cols)

### sql code

In [21]:
input_df = pd.read_csv('/content/gdrive/MyDrive/mimiciv_unzip/hosp/admissions.csv')
# input_df.columns

pd_to_sqlDB(input_df,
            table_name='admissions',
            db_name='default.db')
sql_query_string = """
    SELECT count(distinct subject_id)
    FROM admissions 
"""

result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df

In [31]:
patients = pd.read_csv('/content/gdrive/MyDrive/mimiciv_unzip/hosp/patients.csv')
# patients.columns

pd_to_sqlDB(patients,table_name='patients',db_name='default.db')
sql_query_string = """
    SELECT count(distinct subject_id)
    FROM patients 
"""
result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df

In [60]:
diagnoses_icd = pd.read_csv('/content/gdrive/MyDrive/mimiciv_unzip/hosp/diagnoses_icd.csv')
# diagnoses_icd.columns
# diagnoses_icd['icd_code'].head()
# diagnoses_icd['icd_version'].head()

pd_to_sqlDB(diagnoses_icd,
            table_name='diagnoses_icd',
            db_name='default.db')

# np.sum(diagnoses_icd['icd_version']==9)
# np.sum(diagnoses_icd['icd_code']=='99591') 
# np.sum(diagnoses_icd['icd_code']=='A419') 

sql_query = """ 
  select count(distinct subject_id)
  from icustays 
"""

result_df = sql_query_to_pd(sql_query, db_name='default.db')
result_df # 53569명

In [ ]:
icustays = pd.read_csv('/content/gdrive/MyDrive/mimiciv_unzip/icu/icustays.csv')
pd_to_sqlDB(icustays,
            table_name='icustays',
            db_name='default.db')

#icustays.columns
#icustays


## 환자수 확인

In [74]:
sql_query1 = """ 
  select subject_id
  from diagnoses_icd
  where (icd_version = 9 and icd_code = '99591') or (icd_version = 10 and icd_code = 'A419')
"""
sql_query2 = """ 
  select subject_id
  from icustays
"""

sql_query_to_pd(sql_query1, db_name='default.db')
sql_query_to_pd(sql_query2, db_name='default.db')

sql_query3 = """ 
  select distinct t1.subject_id
  from 
    (select distinct subject_id
    from diagnoses_icd
    where (icd_version = 9 and icd_code = '99591') or (icd_version = 10 and icd_code = 'A419')) as t1
  inner join 
    (select distinct subject_id from icustays) as t2
  on t1.subject_id = t2.subject_id
"""


result_df = sql_query_to_pd(sql_query3, db_name='default.db')

result_df['subject_id'] #4,931명

0       10002013
1       10002428
2       10003400
3       10004401
4       10011365
          ...   
4926    19979469
4927    19981210
4928    19983009
4929    19986715
4930    19997886
Name: subject_id, Length: 4931, dtype: int64

In [ ]:
# 

In [ ]:
#ICU 입실한 환자
#24시간 이내 사망한 환자 제외
#24시간 이내 lab
#spesis 뿐만 아니라 infection 코드가 붙어 있는 사람

# JAMA spesis 정의
# blood 24시간 이내 의심되는 환자 코드 ~
# 항생제를 쓴 사람 ~ 
# 
# 소파 2점 이상


## google big query test

In [7]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [8]:
%load_ext google.colab.data_table

In [12]:
from google.cloud import bigquery

In [13]:
client = bigquery.Client(project='4/0ARtbsJrRvmKJ9YdfG36DUkkIECN4lYSEudwbqY3VilWte6bBI2a8tD3hNAg1B0MPdPOrIQ')

In [14]:
row_count = client.query('''
  SELECT 
    COUNT(*) as total
  FROM `bigquery-public-data.samples.gsod`''').to_dataframe().total[0]

df = client.query('''
  SELECT
    *
  FROM
    `bigquery-public-data.samples.gsod`
  WHERE RAND() < %d/%d
''' % (sample_count, row_count)).to_dataframe()

print('Full dataset has %d rows' % row_count

SyntaxError: ignored

# 새 섹션